# Estimating local and global feature importance scores using DiCE

Summaries of counterfactual examples can be used to estimate importance of features. Intuitively, a feature that is changed more often to generate a proximal counterfactual is an important feature. We use this intuition to build a feature importance score. 

This score can be interpreted as a measure of the **necessity** of a feature to cause a particular model output. That is, if the feature's value changes, then it is likely that the model's output class will also change (or the model's output will significantly change in case of regression model).  

Below we show how counterfactuals can be used to provide local feature importance scores for any input, and how those scores can be combined to yield a global importance score for each feature.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Preliminaries: Loading the data and ML model

In [4]:
from dice_ml.utils import helpers # helper functions
dataset = helpers.load_adult_income_dataset()
helpers.get_adult_data_info()

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

In [5]:
target = dataset["income"]
# Split data into train and test
from sklearn.model_selection import train_test_split
datasetX = dataset.drop("income", axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=0,
                                                    stratify=target)

numerical=["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical) 
from sklearn.compose import ColumnTransformer

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical),
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', LGBMClassifier())])


In [7]:
import dice_ml
from dice_ml import Dice

d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')
model = clf.fit(x_train, y_train)
m = dice_ml.Model(model=model, backend="sklearn")

## Local feature importance

We first generate counterfactuals for a given input point. 

In [21]:
exp = Dice(d, m, method="random")
query_instances = x_train[2:3]
e1 = exp.generate_counterfactuals(query_instances, total_CFs=2)
e1.visualize_as_dataframe(show_only_changes=True)

    age workclass education marital_status    occupation   race  gender  \
0  50.0   Private     Assoc        Married  Professional  White  Female   
1  39.0   Private     Assoc        Married  Professional  White  Female   

   hours_per_week  income  
0            36.0   0.590  
1            85.0   0.671  
0 hours_per_week
    age workclass education marital_status    occupation   race  gender  \
0  50.0   Private     Assoc        Married  Professional  White  Female   
1  39.0   Private     Assoc        Married  Professional  White  Female   

   hours_per_week  income  
0            37.0   0.590  
1            85.0   0.671  
0 hours_per_week
    age workclass education marital_status    occupation   race  gender  \
0  50.0   Private     Assoc        Married  Professional  White  Female   
1  39.0   Private     Assoc        Married  Professional  White  Female   

   hours_per_week  income  
0            38.0   0.590  
1            85.0   0.671  
0 hours_per_week
    age workclass e

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,31,Private,Some-college,Divorced,Blue-Collar,White,Male,40,0.035



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,50.0,-,Assoc,Married,Professional,-,Female,-,0.605
1,39.0,-,Assoc,Married,Professional,-,Female,85.0,0.671


These can now be used to calculate the feature importance scores. 

In [ ]:
imp = exp.feature_importance(query_instances, cf_examples_list=e1.cf_examples_list)
print(imp.local_importance)

Feature importance can also be estimated directly, by leaving the `cf_examples_list` argument blank.

In [ ]:
imp = exp.feature_importance(query_instances, posthoc_sparsity_param=None)
print(imp.local_importance)

## Global importance

For global importance, we need to generate counterfactuals for a representative sample of the dataset. 

In [ ]:
cobj=exp.feature_importance(query_instances[0:10], total_CFs=10, posthoc_sparsity_param=None)
print(cobj.summary_importance)

## Convert the counterfactual output to json

In [ ]:
json_str = cobj.to_json()
print(json_str)

## Convert the json output to a counterfactual object

In [ ]:
imp_r=imp.from_json(json_str)
print([o.visualize_as_dataframe() for o in imp_r.cf_examples_list])
print(imp_r.local_importance)
print(imp_r.summary_importance)